# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# import geoviews as gv
# from geoviews import dim, tile_sources as gvts
# gv.extension('matplotlib')
# gv.output(dpi=120)

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new norfolk,-42.7826,147.0587,13.82,66,50,0.45,AU,1678567945
1,1,punta arenas,-53.1500,-70.9167,15.06,36,0,7.20,CL,1678567946
2,2,esperance,-33.8667,121.9000,15.61,92,30,4.22,AU,1678567947
3,3,saint-philippe,-21.3585,55.7679,24.77,85,99,5.14,RE,1678567948
4,4,arraial do cabo,-22.9661,-42.0278,26.92,69,40,5.66,BR,1678567824


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "ESRI",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[(city_data_df['Max Temp']>=15)&(city_data_df['Max Temp']<=33)&(city_data_df['Humidity']<70)&(city_data_df['Cloudiness']==0)&(city_data_df['Wind Speed']<=3.0)]

# Drop any rows with null values
vacation_df = vacation_df.dropna()

# Display sample data
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,minab,27.1467,57.0801,20.18,68,0,0.00,IR,1678567973
98,98,gerash,27.6652,54.1371,15.08,45,0,1.43,IR,1678567985
198,198,abu dhabi,24.4667,54.3667,25.06,68,0,2.06,AE,1678567965
259,259,nanakuli,21.3906,-158.1547,26.65,64,0,2.57,US,1678568053
286,286,marrakesh,31.6315,-8.0083,26.04,27,0,2.57,MA,1678568065
304,304,vryburg,-26.9566,24.7284,20.20,29,0,2.26,ZA,1678568073
350,350,sao filipe,14.8961,-24.4956,24.47,62,0,2.60,CV,1678568091
367,367,aswan,24.0934,32.9070,22.61,23,0,0.00,EG,1678568058
402,402,alice springs,-23.7000,133.8833,15.75,44,0,2.57,AU,1678568113


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df[['City','Country','Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
69,minab,IR,27.1467,57.0801,68,
98,gerash,IR,27.6652,54.1371,45,
198,abu dhabi,AE,24.4667,54.3667,68,
259,nanakuli,US,21.3906,-158.1547,64,
286,marrakesh,MA,31.6315,-8.0083,27,
304,vryburg,ZA,-26.9566,24.7284,29,
350,sao filipe,CV,14.8961,-24.4956,62,
367,aswan,EG,24.0934,32.9070,23,
402,alice springs,AU,-23.7000,133.8833,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
base_url = "https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:57.0801,27.1467,10000&bias=proximity:57.0801,27.1467&limit=20&apiKey="+geoapify_key
name_address = requests.get(base_url)
# Convert the API response to JSON format
name_address = name_address.json()

In [8]:
name_address['features'][0]['properties']['name']

'هتل صدف'

In [9]:
# Set parameters to search for a hotel
#radius = 10000
params = {'categories':'accommodation.hotel',
          'apiKey': geoapify_key,
          'limit': 20
}
#https:&filter=circle:57.0801,27.1467,10000&bias=proximity:57.0801,27.1467&limit=20&apiKey="+geoapify_key
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row['Lat']
    longitude=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},10000'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    # except Exception as e:
    #     print(e)
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
minab - nearest hotel: هتل صدف
gerash - nearest hotel: هتل آبشار
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
nanakuli - nearest hotel: Aulani, A Disney Resort & Spa
marrakesh - nearest hotel: No hotel found
vryburg - nearest hotel: International Hotel
sao filipe - nearest hotel: Bela Vista
aswan - nearest hotel: Yaseen hotel
alice springs - nearest hotel: Aurora Alice Springs


,City,Country,Lat,Lng,Humidity,Hotel Name
69,minab,IR,27.1467,57.0801,68,هتل صدف
98,gerash,IR,27.6652,54.1371,45,هتل آبشار
198,abu dhabi,AE,24.4667,54.3667,68,Marriott Downtown Abu-Dhabi
259,nanakuli,US,21.3906,-158.1547,64,"Aulani, A Disney Resort & Spa"
286,marrakesh,MA,31.6315,-8.0083,27,No hotel found
304,vryburg,ZA,-26.9566,24.7284,29,International Hotel
350,sao filipe,CV,14.8961,-24.4956,62,Bela Vista
367,aswan,EG,24.0934,32.9070,23,Yaseen hotel
402,alice springs,AU,-23.7000,133.8833,44,Aurora Alice Springs


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 1,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name","City","Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)